### Packages Required

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import ticker
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,minmax_scale
from sklearn.metrics import confusion_matrix,classification_report

from skimage import feature,data,io,measure

In [76]:
import os,random
import itertools
import keras
import PIL
from PIL import Image
from keras import applications
from keras import backend as k
from keras.models import Sequential
from keras.layers import Input,Dense,Activation,Dropout,Conv2D,MaxPooling2D,Flatten
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator,image_utils
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint,Callback,EarlyStopping

### Global Variables

In [11]:
batch_size_train = 70
batch_size_val = 5
batch_size_test = 5
num_classes= 5
STANDARD_SIZE=(224,224)

### Datasets

In [8]:
#Classes
img_class = ['Cheque','Documents','Driving_License','Pancard','Passport']
datagen = ImageDataGenerator(data_format = k.image_data_format())

In [16]:
path = 'C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/'
train_img = ImageDataGenerator().flow_from_directory(path+'Train_Data',target_size=(224,224),classes=img_class,batch_size=batch_size_train)
type(train_img)

Found 70 images belonging to 5 classes.


keras.preprocessing.image.DirectoryIterator

In [17]:
test_img = ImageDataGenerator().flow_from_directory(path+'Test_Data',target_size=(224,224),classes=img_class,batch_size=batch_size_test)
type(test_img)

Found 5 images belonging to 5 classes.


keras.preprocessing.image.DirectoryIterator

In [18]:
val_img = ImageDataGenerator().flow_from_directory(path+'Val_Data',target_size=(224,224),classes=img_class,batch_size=batch_size_val)
type(val_img)

Found 5 images belonging to 5 classes.


keras.preprocessing.image.DirectoryIterator

In [30]:
#Train
train_imgs,train_labels = next(train_img)
print('images_shape : ',train_imgs.shape)
print('labels_shape : ',train_labels.shape)

images_shape :  (70, 224, 224, 3)
labels_shape :  (70, 5)


In [31]:
#Test
test_imgs,test_labels = next(test_img)
print('images_shape : ',test_imgs.shape)
print('labels_shape : ',test_labels.shape)

images_shape :  (5, 224, 224, 3)
labels_shape :  (5, 5)


In [32]:
#val
val_imgs,val_labels = next(val_img)
print('images_shape : ',val_imgs.shape)
print('labels_shape : ',val_labels.shape)

images_shape :  (5, 224, 224, 3)
labels_shape :  (5, 5)


### Model

In [41]:
M_vgg = keras.applications.vgg16.VGG16()
M_vgg.summary()

553467096/553467096 [==============================] - 123s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                          

In [42]:
M_vgg.layers

In [44]:
M0 = Sequential()
for layer in M_vgg.layers:
    M0.add(layer)

In [45]:
M0.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [46]:
for layer in M0.layers:
    layer.trainable = False

In [47]:
M0.add(Dense(5,activation='softmax'))

In [48]:
M0.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [50]:
M0.compile(Adam(learning_rate=0.00015),loss=keras.losses.categorical_crossentropy,metrics=keras.metrics.accuracy)

In [69]:
M0.fit_generator(train_img,validation_data=val_img,epochs=5, verbose=1)

C:\Users\Nithin\AppData\Local\Temp/ipykernel_20788/1574379237.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  M0.fit_generator(train_img,validation_data=val_img,epochs=5, verbose=1)


Epoch 1/5
1/1 [==============================] - 12s 12s/step - loss: 1.5994 - accuracy: 0.0000e+00 - val_loss: 1.5977 - val_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 12s 12s/step - loss: 1.5993 - accuracy: 0.0000e+00 - val_loss: 1.5976 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 12s 12s/step - loss: 1.5992 - accuracy: 0.0000e+00 - val_loss: 1.5975 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 12s 12s/step - loss: 1.5991 - accuracy: 0.0000e+00 - val_loss: 1.5974 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 11s 11s/step - loss: 1.5991 - accuracy: 0.0000e+00 - val_loss: 1.5973 - val_accuracy: 0.0000e+00


In [ ]:
#M0.save_weights('M0.h5')
#M0.load_weights('M0.h5')

In [70]:
M0.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

### Testing Data

In [72]:
test_path = 'C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test'

In [73]:
paths = [test_path+"/cheque copy 2.jpg", test_path+"/driving_license_2 copy 2.jpg",test_path+"/Form1 copy 2.jpg", test_path+"/pan copy 2.jpg", test_path+"/passport copy 2.jpg"]

In [74]:
paths

['C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/cheque copy 2.jpg',
 'C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/driving_license_2 copy 2.jpg',
 'C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/Form1 copy 2.jpg',
 'C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/pan copy 2.jpg',
 'C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/passport copy 2.jpg']

In [78]:
test_x=[]
def convert_to_image(X):
    '''Function to convert all Input Images to the STANDARD_SIZE and create Training Dataset
    '''
    for f in paths:
        #fobj=get_file(f)
        #print(type(fobj))predictions= model.predict(X_test)
        if os.path.isdir(f):
            continue
        img= PIL.Image.open(f)
        img = img.resize(STANDARD_SIZE)
        img= np.array(img)
        X.append(img)
        #print(X_train)
    #print(len(X_train))
    return X
test_x=np.array(convert_to_image(test_x))
datagen.fit(test_x)

In [79]:
pred = M0.predict(test_x)

1/1 [==============================] - 3s 3s/step


In [81]:
test_y = []
for i in range(len(pred)):
    test_y.append(np.argmax(pred[i]))

In [82]:
test_y

[3, 4, 3, 4, 4]

In [85]:
j = 0
for i in test_y:
    print(paths[test_y[j]])
    j = j + 1

C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/pan copy 2.jpg
C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/passport copy 2.jpg
C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/pan copy 2.jpg
C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/passport copy 2.jpg
C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/passport copy 2.jpg


In [91]:
#print(classes_required)
index = img_class.index('Passport')
for i in range(len(test_y)):
    if test_y[i] == index:
        print("Image classified as a Passport document: ", paths[i])

Image classified as a Passport document:  C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/driving_license_2 copy 2.jpg
Image classified as a Passport document:  C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/pan copy 2.jpg
Image classified as a Passport document:  C:/Users/Nithin/Downloads/Image classification_AI_Transfer Learn/Datasets/Ext_Test/passport copy 2.jpg
